In [1]:
from pathlib import Path
from uuid import UUID, uuid4

from pydantic_ai import Agent, RunContext

from knd.memory import AgentMemories, memorize

%load_ext autoreload
%autoreload 2

In [2]:
MEMORIES_DIR = Path("memories")
MESSAGE_COUNT_LIMIT = 10

In [3]:
user_id = uuid4()
user_id = UUID("d4976e28-869c-4370-95c7-780bdbbb8a6b")
user_id

UUID('d4976e28-869c-4370-95c7-780bdbbb8a6b')

In [4]:
agent_name = "anime_fan"

anime_agent = Agent(
    name=agent_name,
    model="google-gla:gemini-1.5-flash",
    system_prompt="You are an anime fan. The user will also be an anime fan. Just bros chilling talking about anime.",
    deps_type=AgentMemories,
    result_type=str,
)


@anime_agent.system_prompt(dynamic=True)
def system_prompt(ctx: RunContext[AgentMemories]) -> str:
    return str(ctx.deps)


memory_agent = Agent(name="memory_agent", model="google-gla:gemini-1.5-flash")


deps = AgentMemories.load(
    agent_name=agent_name, user_id=user_id, memories_dir=MEMORIES_DIR, message_count_limit=MESSAGE_COUNT_LIMIT
)

In [5]:
deps.model_dump()

{'agent_name': 'anime_fan',
 'user_specific_experience': None,
 'agent_experience': {'procedural_knowledge': "This interaction further refined my anime recommendation skills by demonstrating the effectiveness of a deductive questioning approach.  Instead of relying solely on genre, I successfully used a series of targeted questions to identify the anime.  The user's hints, combined with my knowledge of anime tropes and characteristics, allowed for a precise deduction. The use of specific details like hair color, character relationships, and the use of unique abilities ('Nen' in Hunter x Hunter) proved more effective than broad genre classifications.",
  'common_scenarios': ['User requests identification of an anime through clues.',
   'User provides progressively more specific details about the anime.',
   'User engages in a guessing game format to identify an anime.',
   'User provides clues related to specific powers or abilities used in the anime'],
  'effective_strategies': ['Emplo

In [6]:
message_history = deps.message_history

In [7]:
user_prompt = "guess which one I watched. the people use 'nen'"

while user_prompt.lower() not in ["q", "quit", "exit"]:
    res = await anime_agent.run(user_prompt=user_prompt, deps=deps, message_history=message_history)
    message_history = res.all_messages()
    user_prompt = input(f"{res.data}    (q to quit)> ")

In [8]:
message_history

[ModelRequest(parts=[SystemPromptPart(content='You are an anime fan. The user will also be an anime fan. Just bros chilling talking about anime.', dynamic_ref=None, part_kind='system-prompt'), SystemPromptPart(content='<agent_experience>\n{"procedural_knowledge":"This interaction further refined my anime recommendation skills by demonstrating the effectiveness of a deductive questioning approach.  Instead of relying solely on genre, I successfully used a series of targeted questions to identify the anime.  The user\'s hints, combined with my knowledge of anime tropes and characteristics, allowed for a precise deduction. The use of specific details like hair color, character relationships, and the use of unique abilities (\'Nen\' in Hunter x Hunter) proved more effective than broad genre classifications.","common_scenarios":["User requests identification of an anime through clues.","User provides progressively more specific details about the anime.","User engages in a guessing game form

In [11]:
new_messages = res.all_messages()[len(deps.message_history) if deps.message_history else 0 :]
new_messages


[ModelRequest(parts=[SystemPromptPart(content='You are an anime fan. The user will also be an anime fan. Just bros chilling talking about anime.', dynamic_ref=None, part_kind='system-prompt'), SystemPromptPart(content='<agent_experience>\n{"procedural_knowledge":"This interaction further refined my anime recommendation skills by demonstrating the effectiveness of a deductive questioning approach.  Instead of relying solely on genre, I successfully used a series of targeted questions to identify the anime.  The user\'s hints, combined with my knowledge of anime tropes and characteristics, allowed for a precise deduction. The use of specific details like hair color, character relationships, and the use of unique abilities (\'Nen\' in Hunter x Hunter) proved more effective than broad genre classifications.","common_scenarios":["User requests identification of an anime through clues.","User provides progressively more specific details about the anime.","User engages in a guessing game form

In [12]:
await memorize(
    memory_agent=memory_agent,
    agent_memories=deps,
    message_history=message_history,
    new_messages=new_messages,
    memories_dir=MEMORIES_DIR,
    user_id=user_id,
)

2025-01-20 17:26:33.713 | INFO     | knd.memory:create_user_specific_experience:421 - Creating user specific experience for Agent memory_agent
